In [7]:
import os
from model_baselines import BaselineClassifier

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [26]:
#load embeddings
final_arr1 = np.load(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_split_texts' '.npy')
# final_arr2 = np.load(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_split_translated_text' '.npy')
print(final_arr1.shape)
# print(final_arr2.shape)

# final_arr=np.hstack([final_arr1,final_arr2])
final_arr=final_arr1
#load labels for each embeddings
labels_df=pd.read_csv(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_split_labels'+'.csv')
target_names=labels_df['Label'].unique().astype(str).tolist()

labels=labels_df['Label'].values

idx=pd.read_csv(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_and_chich_split_index'+'.csv')
idx=idx['Idx'].values


(8145, 256)


In [30]:
#label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(target_names)

f_target_names=le.transform(target_names)
labels=le.transform(labels)

In [27]:
def get_model(size):
    print('Building model...')
    model = Sequential()
    model.add(Dense(2048, input_shape=(size,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(20))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
def get_results(model,epochs,train_dataset,test_dataset,y_test):
    model.fit(train_dataset,epochs=epochs, batch_size=32,verbose=1)
    print(model.evaluate(test_dataset))
    ans = model.predict(test_dataset)
    print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))
    print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))



In [31]:
X=final_arr
y=labels

skf = StratifiedKFold(n_splits=3,random_state=np.random.seed(42))
skf.get_n_splits(X,y)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np_utils.to_categorical(y_train, 20)
    y_test=np_utils.to_categorical(y_test, 20)
    
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    
    BATCH_SIZE = 32
    SHUFFLE_BUFFER_SIZE = 100

    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    get_results(get_model(256,),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
170/170 [==============================] - 1s 4ms/step - loss: 2.1229 - accuracy: 0.3897
Epoch 2/50
170/170 [==============================] - 1s 4ms/step - loss: 1.7355 - accuracy: 0.4761
Epoch 3/50
170/170 [==============================] - 1s 4ms/step - loss: 1.6166 - accuracy: 0.5024
Epoch 4/50
170/170 [==============================] - 1s 4ms/step - loss: 1.5392 - accuracy: 0.5201
Epoch 5/50
170/170 [==============================] - 1s 4ms/step - loss: 1.4674 - accuracy: 0.5416
Epoch 6/50
170/170 [==============================] - 1s 4ms/step - loss: 1.3953 - accuracy: 0.5545
Epoch 7/50
170/170 [==============================] - 1s 4ms/step - loss: 1.3308 - accuracy: 0.5724
Epoch 8/50
170/170 [==============================] - 1s 4ms/step - loss: 1.2672 - accuracy: 0.5869
Epoch 9/50
170/170 [==============================] - 1s 4ms/step - loss: 1.1952 - accuracy: 0.6127
Epoch 10/50
170/170 [==============================] - 1s 4ms/step - loss: 1.1347 

170/170 [==============================] - 1s 4ms/step - loss: 0.7493 - accuracy: 0.7646
Epoch 18/50
170/170 [==============================] - 1s 4ms/step - loss: 0.6912 - accuracy: 0.7877
Epoch 19/50
170/170 [==============================] - 1s 4ms/step - loss: 0.6385 - accuracy: 0.8064
Epoch 20/50
170/170 [==============================] - 1s 4ms/step - loss: 0.5962 - accuracy: 0.8284
Epoch 21/50
170/170 [==============================] - 1s 4ms/step - loss: 0.5456 - accuracy: 0.8405
Epoch 22/50
170/170 [==============================] - 1s 4ms/step - loss: 0.5039 - accuracy: 0.8604
Epoch 23/50
170/170 [==============================] - 1s 4ms/step - loss: 0.4636 - accuracy: 0.8762
Epoch 24/50
170/170 [==============================] - 1s 4ms/step - loss: 0.4241 - accuracy: 0.8913
Epoch 25/50
170/170 [==============================] - 1s 4ms/step - loss: 0.3853 - accuracy: 0.9076
Epoch 26/50
170/170 [==============================] - 1s 4ms/step - loss: 0.3591 - accuracy: 0.9131
Ep

170/170 [==============================] - 1s 4ms/step - loss: 0.1875 - accuracy: 0.9705
Epoch 34/50
170/170 [==============================] - 1s 4ms/step - loss: 0.1730 - accuracy: 0.9751
Epoch 35/50
170/170 [==============================] - 1s 4ms/step - loss: 0.1616 - accuracy: 0.9757
Epoch 36/50
170/170 [==============================] - 1s 4ms/step - loss: 0.1448 - accuracy: 0.9797
Epoch 37/50
170/170 [==============================] - 1s 4ms/step - loss: 0.1362 - accuracy: 0.9825
Epoch 38/50
170/170 [==============================] - 1s 4ms/step - loss: 0.1269 - accuracy: 0.9820
Epoch 39/50
170/170 [==============================] - 1s 4ms/step - loss: 0.1157 - accuracy: 0.9864
Epoch 40/50
170/170 [==============================] - 1s 4ms/step - loss: 0.1046 - accuracy: 0.9878
Epoch 41/50
170/170 [==============================] - 1s 4ms/step - loss: 0.0948 - accuracy: 0.9904
Epoch 42/50
170/170 [==============================] - 1s 4ms/step - loss: 0.0931 - accuracy: 0.9902
Ep